# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
# first a note about dates...
# dates are such fun

# here i'm testing code to see how to convert the date strings for a films release to an actual date object
import datetime
import numpy as np
import locale

d = datetime.date(2024,10,24)
print(np.datetime64(d)) # actually, we didn't need this step, with a datetime.date object pandas handled it from there

# trouble is, our dates were like: "22 janvier 1975"
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')                       # had to switch locale else english dates not found
my_date = datetime.datetime.strptime('12 juin 2016', '%d %B %Y')
print(np.datetime64(my_date))

# alternatively, we could have handled the french months by setting up a dictionary and mapping it
fr_month_to_int = {'janvier':1, 'février' :2, 'etc': 3}

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import numpy as np
import datetime
from datetime import datetime
import locale

def scrape_allocine(title_type, genre=None, decade=None, year=None):
    """
    title_type = required: series-tv or films
    genre = not implemented - optional: would need to gather which id is for each genre....
    decade = optional: show films from a specific decade
    year = optional: show films from a specific year
    """
    url = f"https://www.allocine.fr/{title_type}/"

    if decade:
        url += 'decennie-'+str(decade)+'/'
        if year:
            url += 'annee-'+str(year)+'/'
    else:
        # url structure requires the decade when specifying a year, if decade is not provided and year is we need
        # to calculate the decade to add it into the url...
        if year:
            url += 'decennie-'+str(year//10*10)+'/'+'annee-'+str(year)+'/'
     response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    movies = []
    
    for div in soup.find_all('div', class_='entity-card-list'):
        # Get movie details

        preview_clip_url = div.find('a').get('href')
        img_url = div.find('img', class_='thumbnail-img').get('src')
        title = div.find('a', class_='meta-title-link').text

        release = div.find('span', class_='date').text.split()
        release_month = div.find('span', class_='date').text.split()[-2]
        release_year = div.find('span', class_='date').text.split()[-1]
        if len(release) > 2:
            release_day = div.find('span', class_='date').text.split()[0]
        else:
            release_day = None

        # using regex here to pull out numericals in this section, to get rid of "h" and "min"
        runtime_hours_mins = re.findall('[0-9]+', div.find('div', class_='meta-body-info').text)

        # calculate the run time in minutes, the above regex may also capture day and year of release
        # so went with -2 and -1 index
        # ...note, would probably be best to include some error handling in case these values are missing
        runtime_mins = int(runtime_hours_mins[-2])*60 + int(runtime_hours_mins[-1])

        director_section = div.find('div', class_='meta-body-direction')
        director = director_section.find_all('span')[1].text
        
                
        # Create a dictionary with movie details
        movie_details = {'title': title, 'image_url': img_url, 'preview_clip_url': preview_clip_url, 
                         'release_day' : release_day, 'release_month':release_month, 'release_year':release_year,
                         'runtime_mins':runtime_mins, 'director': director}
        
        # Append the dictionary to the movies list
        movies.append(movie_details)
    
    # Create a DataFrame from the movies list
    df = pd.DataFrame(movies)
    return df

# Get the DataFrame by calling the function with the desired parameters
df = scrape_allocine(title_type='films')

# fix dates, we could avoid the lambda here by adding these as extra columns in the dictionary...

locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
df['release_date_fr'] = df['release_day'].apply(lambda x:x if x is not None else '1') + " " + df['release_month'] + " " + df['release_year']
df['release_date'] = df['release_date_fr'].apply(lambda x:datetime.strptime(x, '%d %B %Y'))

# Display the DataFrame
df

In [ ]:
df.dtypes

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
from time import sleep

def generate_url(url, page):
    return url + "?page=" + str(page)

def scrape_allocine(title_type, genre=None, decade=None, year=None, delay=3, max_pages=3):
    """
    title_type = required: series-tv or films
    genre = not implemented - optional: would need to gather which id is for each genre....
    decade = optional: show films from a specific decade
    year = optional: show films from a specific year
    """
    url = f"https://www.allocine.fr/{title_type}/"

    if decade:
        url += 'decennie-'+str(decade)+'/'
        if year:
            url += 'annee-'+str(year)+'/'
    else:
        # url structure requires the decade when specifying a year, if decade is not provided and year is we need
        # to calculate the decade to add it into the url...
        if year:
            url += 'decennie-'+str(year//10*10)+'/'+'annee-'+str(year)+'/'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    pagination = soup.find('div',class_='pagination-item-holder')
    pages = pagination.find_all('span')
    total_pages = int(pages[-1].text)

    movies = []
    for page in range(min([total_pages, max_pages])):

        url = generate_url(url, page+1)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        for div in soup.find_all('div', class_='entity-card-list'):
            # Get movie details

            preview_clip_url = div.find('a').get('href')
            img_url = div.find('img', class_='thumbnail-img').get('src')
            title = div.find('a', class_='meta-title-link').text

            release = div.find('span', class_='date').text.split()
            release_month = div.find('span', class_='date').text.split()[-2]
            release_year = div.find('span', class_='date').text.split()[-1]
            if len(release) > 2:
                release_day = div.find('span', class_='date').text.split()[0]
            else:
                release_day = None

            # using regex here to pull out numericals in this section, to get rid of "h" and "min"
            runtime_hours_mins = re.findall('[0-9]+', div.find('div', class_='meta-body-info').text)

            # calculate the run time in minutes, the above regex may also capture day and year of release
            # so went with -2 and -1 index
            # ...note, would probably be best to include some error handling in case these values are missing
            runtime_mins = int(runtime_hours_mins[-2])*60 + int(runtime_hours_mins[-1])

            director_section = div.find('div',class_='meta-body-direction')
            director = director_section.find_all('span')[1].text
            
                    
            # Create a dictionary with movie details
            movie_details = {'title': title, 'image_url': img_url, 'preview_clip_url': preview_clip_url, 
                            'release_day' : release_day, 'release_month':release_month, 'release_year':release_year,
                            'runtime_mins':runtime_mins, 'director':director}
            
            # Append the dictionary to the movies list
            movies.append(movie_details)

        sleep(delay)
    
    # Create a DataFrame from the movies list
        df = pd.DataFrame(movies)
    return df

# Get the DataFrame by calling the function with the desired parameters
df = scrape_allocine(title_type='films', decade=1970, year=1975)

# fix dates, we could avoid the lambda here by adding these as extra columns in the dictionary...
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
df['release_date_fr'] = df['release_day'].apply(lambda x:x if x is not None else '1') + " " + df['release_month'] + " " + df['release_year']
df['release_date'] = df['release_date_fr'].apply(lambda x:datetime.strptime(x, '%d %B %Y'))

# Display the DataFrame
df

In [ ]:
# used while figuring out what bits of the soup to use...

url = 'https://www.allocine.fr/films/decennie-1970/annee-1975/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
div = soup.find_all('div',class_='entity-card-list')
director_section = div[2].find('div', class_='meta-body-direction')
print(director_section)
director = director_section.find_all('span')[1].text
director

# director wasn't coming through as expected as first, when inspecting the was in link ('a' tag) though
# when i checked here, i saw it was in a span tag